In [1]:
import regex as re
import pandas as pd
import numpy as np
import nltk
from nltk.collocations import *
from scipy.stats import spearmanr

In [2]:
df = pd.read_csv('court-V-N.csv') # corpus
rows = list(map(list, df.values)) # list of rows in corpus
bigram_measures = nltk.collocations.BigramAssocMeasures()
finder = BigramCollocationFinder.from_documents(rows)

Я буду использовать метрики **likelihood ratio** и **jaccard**

In [6]:
bigram_measures.jaccard.__doc__

'Scores ngrams using the Jaccard index.'

In [7]:
top_jaccard = finder.nbest(bigram_measures.jaccard, 10)

In [19]:
bigram_measures.likelihood_ratio.__doc__

'Scores ngrams using likelihood ratios as in Manning and Schutze 5.3.4.\n        '

In [20]:
top_lr = finder.nbest(bigram_measures.likelihood_ratio, 10)

In [22]:
list_with_scores_lr = list(finder.score_ngrams(bigram_measures.likelihood_ratio))
list_no_scores_lr = [(x[0][0].strip(' '), x[0][1].strip(' ')) for x in list_with_scores_lr]
list_with_scores_jaccard = list(finder.score_ngrams(bigram_measures.jaccard))
list_no_scores_jaccard = [(x[0][0].strip(' '), x[0][1].strip(' ')) for x in list_with_scores_jaccard]

In [58]:
gold_st = [(("ПРИНЯТЬ", "РЕШЕНИЕ"),1), (("ВЫНЕСТИ", "ПРИГОВОР"),2), (("УДОВЛЕТВОРИТЬ", "ИСК"),3), 
                (("ПРИЗНАТЬ", "ВИНОВНАЯ"),4), (("ПРИГОВОРИТЬ", "ЗАКЛЮЧЕНИЕ"),5), 
                (("ПРОДЛИТЬ", "СРОК"),6), (("ОБЖАЛОВАТЬ", "ПРИГОВОР"),7), (("ПРЕКРАТИТЬ", "ДЕЛО"),8), 
                (("ОТКЛОНИТЬ", "ИСК"),9), (("ПРИЗНАТЬ", "СООБЩНИК"),10)]

In [59]:
ranks_golden = [p[1] for p in gold_st]
ranks_lr = [list_no_scores_lr.index(i[0]) if i[0] in list_no_scores_lr else None for i in gold_st]
ranks_jaccard = [list_no_scores_jaccard.index(i[0]) if i[0] in list_no_scores_jaccard else None for i in gold_st]
# print('gold', list(zip(ranks_golden, gold_st)))
print('Likelihood Ratio:')
print('\n'.join(map(str, list(zip(ranks_lr, gold_st)))))
print('\n')
print('Jaccard:')
print('\n'.join(map(str, list(zip(ranks_jaccard, gold_st)))))


Likelihood Ratio:
(5, (('ПРИНЯТЬ', 'РЕШЕНИЕ'), 1))
(86, (('ВЫНЕСТИ', 'ПРИГОВОР'), 2))
(3, (('УДОВЛЕТВОРИТЬ', 'ИСК'), 3))
(12, (('ПРИЗНАТЬ', 'ВИНОВНАЯ'), 4))
(2250, (('ПРИГОВОРИТЬ', 'ЗАКЛЮЧЕНИЕ'), 5))
(193, (('ПРОДЛИТЬ', 'СРОК'), 6))
(2123, (('ОБЖАЛОВАТЬ', 'ПРИГОВОР'), 7))
(1648, (('ПРЕКРАТИТЬ', 'ДЕЛО'), 8))
(80, (('ОТКЛОНИТЬ', 'ИСК'), 9))
(2146, (('ПРИЗНАТЬ', 'СООБЩНИК'), 10))


Jaccard:
(903, (('ПРИНЯТЬ', 'РЕШЕНИЕ'), 1))
(1249, (('ВЫНЕСТИ', 'ПРИГОВОР'), 2))
(772, (('УДОВЛЕТВОРИТЬ', 'ИСК'), 3))
(1006, (('ПРИЗНАТЬ', 'ВИНОВНАЯ'), 4))
(1978, (('ПРИГОВОРИТЬ', 'ЗАКЛЮЧЕНИЕ'), 5))
(1005, (('ПРОДЛИТЬ', 'СРОК'), 6))
(1847, (('ОБЖАЛОВАТЬ', 'ПРИГОВОР'), 7))
(1899, (('ПРЕКРАТИТЬ', 'ДЕЛО'), 8))
(1369, (('ОТКЛОНИТЬ', 'ИСК'), 9))
(2202, (('ПРИЗНАТЬ', 'СООБЩНИК'), 10))


In [54]:
# топ-коллокации по метрике loglikelihood ratio
top_lr

[('СУД ', 'ПРИЗНАТЬ '),
 ('ВЫДАТЬ ', 'САНКЦИЯ '),
 ('СУД ', 'УДОВЛЕТВОРИТЬ '),
 ('УДОВЛЕТВОРИТЬ ', 'ИСК '),
 ('УДОВЛЕТВОРИТЬ ', 'ХОДАТАЙСТВО '),
 ('ПРИНЯТЬ ', 'РЕШЕНИЕ '),
 ('САНКЦИЯ ', 'АРЕСТ '),
 ('МЕРА ', 'ПРЕСЕЧЕНИЕ '),
 ('НАЛОЖИТЬ ', 'АРЕСТ '),
 ('СУД ', 'ВЫНЕСТИ ')]

In [55]:
# топ-коллокации по метрике jaccard
top_jaccard

[(' АКЦИОНЕР ', 'РОЗНИЧНЫЙ '),
 (' АРМИЯ ', 'ВОССТАНОВИТЬСЯ '),
 (' БОЛЬШИНСТВО ', 'СООТЕЧЕСТВЕННИК '),
 (' БРИГАДИР ', 'ВИКТОР '),
 (' ВЕСТИ ', 'ПЕРЕГОВОРЫ '),
 (' ВЗЯТЬ ', 'РЕАЛИЗАЦИЯ '),
 (' ВОЗГЛАВИТЬ ', 'СПИСОК '),
 (' ВРЕМЯ ', 'КНЯЗЕВ '),
 (' ВСПОМИНАТЬСЯ ', 'ПОПЫТКА '),
 (' ДОЛЛАР ', 'ПОТРАТИТЬ ')]

In [50]:
from nltk.metrics.spearman import *

In [51]:
ranks_golden = np.array(ranks_golden, dtype=np.float)
ranks_lr = np.array(ranks_lr, dtype=np.float)
ranks_jaccard = np.array(ranks_jaccard, dtype=np.float)

In [52]:
spearmanr(ranks_golden,ranks_lr, nan_policy='omit')

SpearmanrResult(correlation=0.58787878787878778, pvalue=0.07387770688865801)

In [53]:
spearmanr(ranks_golden,ranks_jaccard, nan_policy='omit')

SpearmanrResult(correlation=0.70909090909090899, pvalue=0.021665923367532435)

## Отчёт
0. Корпус с судебной тематикой, лемматизированный, очищен от стоп-слов (местоимения, предлоги и т.д.), объем - 5.100 словоупотреблений
1. **Likelihood ratio.** Не вошли в золотой стандарт:
     + ('СУД ', 'ПРИЗНАТЬ ') - полезная коллокация
     + ('ВЫДАТЬ ', 'САНКЦИЯ ')
     + ('СУД ', 'УДОВЛЕТВОРИТЬ ') - полезная коллокация
     + ('УДОВЛЕТВОРИТЬ ', 'ХОДАТАЙСТВО ') - полезная коллокация
     + ('САНКЦИЯ ', 'АРЕСТ '),
     + ('МЕРА ', 'ПРЕСЕЧЕНИЕ '), - полезная коллокация (но часть речи нам не подходит)
     + ('НАЛОЖИТЬ ', 'АРЕСТ ') - полезная коллокация
     + ('СУД ', 'ВЫНЕСТИ ') - полезная коллокация
2. **Jaccard** - в золотой стандарт не вошла ни одна из выделенных данным методом коллокаций.    
    Ошибочными на мой взляд являются коллокации:
     + (' БРИГАДИР ', 'ВИКТОР '),
     + (' ВЗЯТЬ ', 'РЕАЛИЗАЦИЯ '),
     + (' ВРЕМЯ ', 'КНЯЗЕВ '),
